# Coursera Week04 Assignment

## Using BeautifulSouf we will first scrapt the wikipedia url to get the list of postal codes

In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from urllib.request import urlopen

We will use beautifulsoup to filter a request on the wikipedia url

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
html = urlopen(url)
soup = BeautifulSoup(html, 'html')

Now, walk through the lines of the table to create tree lists with PostalCode, Borough and Neighborhood

In [3]:
sigla, borough, Neigh = [], [], []
old_postal, old_neigh, old_boro = None, '', ''
for tr in soup.find_all('tr'):
    b = tr.text.split('\n')[1:]
    try:
        # Check if the <tr> is indeed from the wanted table
        if b[0][0] == 'M':
            # the next if statment for the requeriment: 
            # "Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned."
            if b[1] != 'Not assigned':
                # now check if the postalcode is not repeted:
                if b[0] != old_postal and old_postal != None:
                    # add to the list (table)
                    sigla.append(old_postal)
                    # if not assigned to neighborhood, give borough value
                    if old_neigh == 'Not assigned':
                        old_neigh = old_boro
                    borough.append(old_boro)
                    Neigh.append(old_neigh)
                    # now clean the temporary neighborhood
                    old_neigh = ''
                if old_neigh != '':
                    old_neigh += ', '
                old_neigh += b[2]
                old_boro = b[1]
                old_postal = b[0]
                    
    except:
        pass

Now put those lists on the dataframe

In [4]:
df = pd.DataFrame(list(zip(sigla, borough, Neigh)), columns=['PostalCode', 'Borough', 'Neighborhood'])
df.head(20)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge, Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens, Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson, Garden District"


Let's see the shape:

In [5]:
df.shape

(102, 3)

Now, to get the geolocation from the postal code:

In [6]:
import geocoder

Code not working as google api does not acept calls withot registered key no more

In [7]:
# lat_lng_coords = None
# lat, long = [], []
# for postalCode in df.PostalCode.values:
#     # loop until you get the coordinates
#     while(lat_lng_coords is None):
#       g = geocoder.google(f'{postalCode}, Toronto, Ontario')
#       lat_lng_coords = g.latlng
#     lat.append(lat_lng_coords[0])
#     long.append(lat_lng_coords[1])

Using the data on the csv file as a workoroung as requested:

In [8]:
# getting the data from the url to a pandas dataframe
url = 'http://cocl.us/Geospatial_data'
geo_df = pd.read_csv(url)

In [9]:
# renaming Postal code colum to match previous df:
geo_df.rename(columns={"Postal Code": "PostalCode"}, inplace=True)

In [10]:
# check if everything is correct:
geo_df.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Now, let's merge the dataframes:

In [11]:
full = pd.merge(df, geo_df, on='PostalCode')

In [12]:
full.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937


In [13]:
full.shape

(102, 5)

Some analysis of the data and visual plot of the map

In [36]:
from geopy.geocoders import Nominatim
import folium
import folium.plugins as plugins
import requests

In [15]:
# Ltes create a map of toronto with the given areas code latitude and longitude:
map_toronto = folium.Map(location=[43.6542599, -79.3606359], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(full['Latitude'], full['Longitude'], full['Borough'], full['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  

In [16]:
map_toronto

Now some infom from 4squere

In [17]:
manhattan_data = full[full['Borough'] == 'Toronto'].reset_index(drop=True)
manhattan_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude


In [18]:
CLIENT_ID = 'DVUYZ0B5CPKNIIXLYAJMGRKZBYUP1U20PUC0AKEAFQT0NDLJ' # your Foursquare ID
CLIENT_SECRET = 'G5W51BKU2JQDRPVKLNMOHLW0L2NBVYQZ3HUZXVXQ14TKEKGO' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: DVUYZ0B5CPKNIIXLYAJMGRKZBYUP1U20PUC0AKEAFQT0NDLJ
CLIENT_SECRET:G5W51BKU2JQDRPVKLNMOHLW0L2NBVYQZ3HUZXVXQ14TKEKGO


In [19]:
def get_info(lat, lng):
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        lat, lng, 
        100, 
        500)
    results = requests.get(url).json()
    return results['response']['groups'][0]['items']

Lets see how the data is formated:

In [20]:
venues = get_info(43.6542599, -79.3606359)

In [21]:
venues

[{'reasons': {'count': 0,
   'items': [{'summary': 'This spot is popular',
     'type': 'general',
     'reasonName': 'globalInteractionReason'}]},
  'venue': {'id': '50760559e4b0e8c7babe2497',
   'name': 'Body Blitz Spa East',
   'location': {'address': '497 King Street East',
    'crossStreet': 'btwn Sackville St and Sumach St',
    'lat': 43.65473505045365,
    'lng': -79.35987433132891,
    'labeledLatLngs': [{'label': 'display',
      'lat': 43.65473505045365,
      'lng': -79.35987433132891}],
    'distance': 80,
    'postalCode': 'M5A 1L9',
    'cc': 'CA',
    'city': 'Toronto',
    'state': 'ON',
    'country': 'Canada',
    'formattedAddress': ['497 King Street East (btwn Sackville St and Sumach St)',
     'Toronto ON M5A 1L9',
     'Canada']},
   'categories': [{'id': '4bf58dd8d48988d1ed941735',
     'name': 'Spa',
     'pluralName': 'Spas',
     'shortName': 'Spa',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/spa_',
      'suffix': '.png'},
     'pri

In [22]:
venues[0]['venue']['categories']

[{'id': '4bf58dd8d48988d1ed941735',
  'name': 'Spa',
  'pluralName': 'Spas',
  'shortName': 'Spa',
  'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/spa_',
   'suffix': '.png'},
  'primary': True}]

In [23]:
venues[0]['venue']['categories'][0]['name']

'Spa'

Now to the real work:

Let's walk through all the locations and count how many are there in every point

In [37]:
places = []
for lat, lng in zip(full['Latitude'], full['Longitude']):
    venues = get_info(lat, lng)
    n = 0
    for venue in venues:
        for v in venue['venue']['categories']:
            print(v['name'])
            n += 1
    places.append(n)
    print('New location: ', lat, lng, n)

New location:  43.7532586 -79.3296565 0
New location:  43.725882299999995 -79.31557159999998 0
Spa
Breakfast Spot
Art Gallery
New location:  43.6542599 -79.3606359 3
Clothing Store
Hot Dog Joint
Clothing Store
Bridal Shop
New location:  43.718517999999996 -79.46476329999999 4
New location:  43.6623015 -79.3894938 0
New location:  43.6678556 -79.53224240000002 0
New location:  43.806686299999996 -79.19435340000001 0


ConnectionError: HTTPSConnectionPool(host='api.foursquare.com', port=443): Max retries exceeded with url: /v2/venues/explore?&client_id=DVUYZ0B5CPKNIIXLYAJMGRKZBYUP1U20PUC0AKEAFQT0NDLJ&client_secret=G5W51BKU2JQDRPVKLNMOHLW0L2NBVYQZ3HUZXVXQ14TKEKGO&v=20180605&ll=43.745905799999996,-79.352188&radius=100&limit=500 (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x000001B8E65B1320>: Failed to establish a new connection: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond',))

In [35]:
map_toronto = folium.Map(location=[43.6542599, -79.3606359], zoom_start=10)
map_toronto.add_children(plugins.HeatMap(zip(full['Latitude'], full['Longitude'], places), radius = 10))

map_toronto

C:\Users\Gustavo\Anaconda2\envs\tf3\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: Method `add_children` is deprecated. Please use `add_child` instead.
  
